## Build A Motion Detected Alarm System with Python and OpenCV

### Introduction

In this exercise we are going to develop – again! – a motion triggered alarm based on the frame differencing method. Now, we are going to do this using Python and OpenCV.

Open Source Computer Vision Library (OpenCV) is a library written in C/C++ mainly aimed at real-time computer vision.

OpenCV supports a wide variety of programming languages such as C++, Python, Java, etc. 

In particular, in this exercise we are going to introduce OpenCV-Python, a Python wrapper for the original OpenCV C++ implementation.


The approach for developing this application is the same as proposed in Exercise 14:

When the program starts, the program will capture from the webcam the ‘baseline’ image (see the left image below). This is the image without any 'intruder'. The program will keep capturing frames and comparing them with the baseline image. If no one enters the frame, nothing will happen. However, when somebody or something enters the frame (see the right image below), the program will detect that the captured frame and the baseline image are ‘very’ different and will trigger the audio alarm.

![Alarm](../Images/Alarm_v2.png)

### Exercises

**Very important: you need to run the code in your local machine for displaying the video and using the webcam.**

**Exercise 1.1**: Download (or copy) this Jupyter notebook into your local machine.

**Exercise 1.2**: Install the required Python libraries

                pip install pyttsx3
                pip install pywin32
                pip install numpy
                pip install opencv-python

**Exercise 1.3**: Run and study the code *Capture and display the frames of a video with Python and OpenCV*.

**Exercise 1.4**: Run and study the code *Capture and display webcam with Python and OpenCV*.

**Exercise 1.5**: Run and study the code *Build a motion detected alarm system with Python*.

### Exercise 1.3. Capture and display the frames of a video with Python and OpenCV.

 Run and study the code Capture and display the frames of a video with Python and OpenCV.

In [ ]:
!pip install pyttsx3
!pip install pywin32
!pip install numpy
!pip install opencv-python

In [ ]:
# Capture and display the frames of a video with Python and OpenCV

import cv2

# Create a VideoCapture object and read the frames from an input file
# You can download this video from https://www.loc.gov/item/00694162/ (or use another video!)
# video=cv2.VideoCapture('media-ammem-AmericaAtWorkAmericaAtLeisure-1968.mp4')
# video=cv2.VideoCapture('http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4')
video = cv2.VideoCapture('./butterfly_flower_insect_nature_515.mp4')


# Check if the video opened successfully
if (video.isOpened()== False): 
  print("Error opening video file")

# # Read until video is completed or we press 'q'
while True:
    # Capture frame-by-frame
    # Note that VideoCapture captures the frames of a video without considering the fps of the video
    check, frame = video.read()

    if check == True:
    
        cv2.imshow("movie",frame)
    
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Exercise 1.4. Capture and display webcam with Python and OpenCV.

Run and study the code Capture and display webcam with Python and OpenCV.

In [ ]:
# Capture and display webcam with Python and OpenCV

import cv2

# We use VideoCapture function to create the video capture object
# Note we put '0' to capture webcam
video=cv2.VideoCapture(0)

# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while True:
    check, frame = video.read()
    cv2.imshow(q"Webcam",frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# After the loop release the video object
video.release()

# Destroy all the windows
cv2.destroyAllWindows()

### Exercise 1.5. Build a motion detected alarm system with Python.

Run and study the code Build a motion detected alarm system with Python.

In particular, analyse and try different parameters in the functions:

- GaussianBlur(): for noise reduction (smoothening).
- threshold(): you need to change the threshold function values for a better performance with your webcam , room's light, etc.
- findContours(): this method identifies all the contours in an image. Experiment with the different contour retrieval modes (https://docs.opencv.org/4.5.2/d3/dc0/group__imgproc__shape.html#ga819779b9857cc2f8601e6526a3a5bc71) and contour approximation methods (https://docs.opencv.org/4.5.2/d3/dc0/group__imgproc__shape.html#ga4303f45752694956374734a03c54d5ff),

In [ ]:
# Build A Motion Detected Alarm System with Python

# let’s import the libraries
# For playing the audio, we will be using “pyttsx3” python library to convert text to speech
import cv2
import pyttsx3
import threading
import time


# This funtion plays the audio message
# def voice_alarm(alarm_sound):
#     alarm_sound.say("Intruder Detected")
#     alarm_sound.runAndWait()
    

# Setting parameters for voice
# alarm_sound = pyttsx3.init()
# voices = alarm_sound.getProperty('voices')
# alarm_sound.setProperty('voice', voices[0].id)
# alarm_sound.setProperty('rate', 150)


# The function to play the audio wil be executed in a separate thread.
# So, there won't be lag in the video feed while the audio alert message is playing.
# alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))


#status_list=[None,None]
status_list=[0,0]
initial_frame = None


# We use VideoCapture function to create the video capture object
video=cv2.VideoCapture(0)


# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while True:
    check, frame = video.read()
    status=0

    # Gray conversion and noise reduction (smoothening)
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur_frame=cv2.GaussianBlur(gray_frame,(25,25),0)

    
    # The first captured frame is the baseline image
    if initial_frame is None:
        initial_frame = blur_frame
        continue

    # The difference between the baseline and the new frame
    delta_frame=cv2.absdiff(initial_frame,blur_frame)
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(delta_frame,180,255, cv2.THRESH_BINARY)[1]
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # contourArea() method filters out any small contours
        # You can change this value
        if cv2.contourArea(c) < 1000:
            continue
        status=status + 1
        (x, y, w, h)=cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    status_list.append(status)


    # The alarm is triggered if an 'intruder' is detected
    # We can also trigger the alarm only if a moving object is detected with
    #if status_list[-1]>= 1 and status_list[-2]==0:    
#     if status_list[-2]>= 1:
#         if (alarm.is_alive() == False):
#             alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))
#             alarm.start()

        
    # To better understand the application, we can visualise the different frames generated
    cv2.imshow('Webcam', frame)
    cv2.imshow('Baseline image', initial_frame)
    cv2.imshow("Gray Frame",gray_frame)
    cv2.imshow('Delta frame', delta_frame)   
    cv2.imshow('Threshold frame', threshold_frame)
    

    # Stop the program by pressing 'q'    q
    if cv2.waitKey(1) == ord('q'):
        break

        
# After the loop release the video object, stop the alarm
# and destroy all the windows
# alarm_sound.stop()
video.release()
cv2.destroyAllWindows()

Note: This lab is based on https://github.com/arindomjit/Motion_Detected_Alarm by Arindomjit Bhattacharjee.

In [ ]:
# Build A Motion Detected Alarm System with Python

# let’s import the libraries
# For playing the audio, we will be using “pyttsx3” python library to convert text to speech
import cv2
import pyttsx3
import threading
import time


# This funtion plays the audio message
# def voice_alarm(alarm_sound):
#     alarm_sound.say("Intruder Detected")
#     alarm_sound.runAndWait()
    

# Setting parameters for voice
# alarm_sound = pyttsx3.init()
# voices = alarm_sound.getProperty('voices')
# alarm_sound.setProperty('voice', voices[0].id)
# alarm_sound.setProperty('rate', 150)


# The function to play the audio wil be executed in a separate thread.
# So, there won't be lag in the video feed while the audio alert message is playing.
# alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))


#status_list=[None,None]
status_list=[0,0]
initial_frame = None


# We use VideoCapture function to create the video capture object
video=cv2.VideoCapture('Exercise1_Files/Traffic_Laramie_1.mp4')


# We start an infinite loop and keep reading frames from the webcam until we press 'q'
while True:
    check, frame = video.read()
    status=0
    height, width , _ = frame.shape


    main_street = frame[int(height/2): height, 0:width]



    # Gray conversion and noise reduction (smoothening)
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur_frame=cv2.GaussianBlur(gray_frame,(25,25),0)

    
    # The first captured frame is the baseline image
    if initial_frame is None:
        initial_frame = blur_frame
        continue

    # The difference between the baseline and the new frame
    delta_frame=cv2.absdiff(initial_frame,blur_frame)
    # The difference (the delta_frame) is converted into a binary image
    # If a particular pixel value is greater than a certain threshold (specified by us here as 150),
    # it will be assigned the value for White (255) else Black(0)
    # Important: you may have to change the threshold value for a better performance with your webcam , room's light, etc.
    threshold_frame=cv2.threshold(delta_frame,50,255, cv2.THRESH_BINARY)[1]
    

    # The cv2.findContours() method we will identify all the contours in our image.
    # This method expects 3 parameters, (a) image, (b) contour retrieval mode and
    # (c) contour approximation method
    (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # contourArea() method filters out any small contours
        # You can change this value
        if cv2.contourArea(c) < 1000:
            continue
        status=status + 1
        (x, y, w, h)=cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)
    status_list.append(status)


    # The alarm is triggered if an 'intruder' is detected
    # We can also trigger the alarm only if a moving object is detected with
    #if status_list[-1]>= 1 and status_list[-2]==0:    
#     if status_list[-2]>= 1:
#         if (alarm.is_alive() == False):
#             alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))
#             alarm.start()

        
    # To better understand the application, we can visualise the different frames generated
    cv2.imshow('Webcam', frame)
    cv2.imshow('Baseline image', initial_frame)
    cv2.imshow("Gray Frame",gray_frame)
    cv2.imshow('Delta frame', delta_frame)   
    cv2.imshow('Threshold frame', threshold_frame)
    cv2.imshow('Main Street', main_street)
    

    # Stop the program by pressing 'q'    q
    if cv2.waitKey(1) == ord('q'):
        break

        
# After the loop release the video object, stop the alarm
# and destroy all the windows
# alarm_sound.stop()
video.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

cap=cv2.VideoCapture('Exercise1_Files/Traffic_Laramie_1.mp4')


# Capture the first frame and use it as the background image
_, bg = cap.read()
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)

while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     print(height, width)

    # Perform background subtraction to obtain the foreground mask
    fgbg = cv2.createBackgroundSubtractorMOG2()
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Apply the final mask on the current frame to obtain the foreground objects
    res = cv2.bitwise_and(frame, frame, mask=mask)


    # Update the background image for the next frame
    bg = gray

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_1.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
height, width , _ = bg.shape
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)

bg = bg[int(height/2): height, 0:width]


# Initialize list to hold trackers for each car
trackers = []

while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width , _ = frame.shape
    main_street = frame[int(height/2): height, 0:width]
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
    
    
    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Apply morphological operations to improve mask quality
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        if cv2.contourArea(cnt) < 2000:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h
        if area > 7000 and aspect_ratio > 0.3 and aspect_ratio < 3:


#         (x, y, w, h)=cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y + int(height/2)), (x+w, y+h + int(height/2)), (0,255,0), 1)

#         # Filter based on size, aspect ratio, and position
#         if area > 500 and aspect_ratio > 0.3 and aspect_ratio < 3 and y > 100 and y < 400:
#             # Initialize a new tracker for the car
#             tracker = cv2.TrackerCSRT_create()
#             tracker.init(frame, (x, y, w, h))
#             trackers.append(tracker)

    # Update the tracker for each car and draw the bounding box
#     for tracker in trackers:
#         ok, bbox = tracker.update(frame)
#         if ok:
#             x, y, w, h = [int(v) for v in bbox]
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the result
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Main street', main_street)

    # Update the background image for the next frame
    bg = gray

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# traffic counter

In [ ]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_2.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
height, width , _ = bg.shape

x_offset = 150
y_offset = 170
x_start_point = int(width/2) - x_offset
x_end_point = int(width/2 )
y_start_point = int(height/2)
y_end_point = int(height/2) + y_offset
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)

bg = bg[y_start_point: y_end_point,x_start_point:x_end_point]


    trackers = []

# Initialize multi-object tracker
tracker = cv2.legacy.MultiTracker_create()

# Initialize variables to keep track of cars
car_count = 0
cars_passed = set()


while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width , _ = frame.shape
    city_center_street = frame[int(height/2): (int(height/2) + y_offset), int(width/2) - x_offset:int(width/2 )]
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(city_center_street, cv2.COLOR_BGR2GRAY)
    
    
    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Apply morphological operations to improve mask quality
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        if cv2.contourArea(cnt) < 2000:
            continue
            
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h
        if area > 7000 and aspect_ratio > 0.3 and aspect_ratio < 3:


#         (x, y, w, h)=cv2.boundingRect(c)
            x=  x + x_start_point
            y = y + y_start_point
           
            

            cv2.rectangle(frame, (x, y ), (x+w, y+h), (0,255,0), 1)
            
            bbox = (x , y , w, h)
            tracker.add(cv2.legacy.TrackerCSRT_create(), frame, bbox)
    print(tracker.update(frame))
    for i, newbox in enumerate(tracker.update(frame)):
        x, y, w, h = [int(v) for v in newbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Check if the object has passed the line and increment the car count
        if x + w // 2 > x2 and i not in cars_passed:
            car_count += 1
            cars_passed.add(i)

#         # Filter based on size, aspect ratio, and position
#         if area > 500 and aspect_ratio > 0.3 and aspect_ratio < 3 and y > 100 and y < 400:
#             # Initialize a new tracker for the car
#             tracker = cv2.TrackerCSRT_create()
#             tracker.init(frame, (x, y, w, h))
#             trackers.append(tracker)

    # Update the tracker for each car and draw the bounding box
#     for tracker in trackers:
#         ok, bbox = tracker.update(frame)
#         if ok:
#             x, y, w, h = [int(v) for v in bbox]
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.rectangle(frame, (int((x_start_point + x_end_point)/2), y_start_point  ), (int((x_start_point + x_end_point)/2), y_end_point), (0,255,0), 1)
    # Show the result
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow(' Counting point', city_center_street)
    cv2.putText(frame, f"Cars passed: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Update the background image for the next frame
    bg = gray

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindos()



In [ ]:
import cv2

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[5]

if tracker_type == 'BOOSTING':
    tracker = cv2.legacy.TrackerBoosting_create()
if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create() 
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create() 
if tracker_type == 'TLD':
    tracker = cv2.legacy.TrackerTLD_create() 
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create() 
if tracker_type == 'MOSSE':
    tracker = cv2.legacy.TrackerMOSSE_create()
if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

In [ ]:
tracker = cv2.legacy.TrackerBoosting_create()

In [ ]:
tracker = cv2.TrackerMIL

In [ ]:
tracker.update

In [ ]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_2.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)

# Initialize list to hold trackers for each car
trackers = []

while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Apply morphological operations to improve mask quality
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h

        # Filter based on size, aspect ratio, and position
        if area > 6000 and aspect_ratio > 0.3 and aspect_ratio < 3 and y > 100 and y < 400:
            # Initialize a new tracker for the car
            tracker = cv2.legacy.TrackerCSRT_create()
            tracker.init(frame, (x, y, w, h))
            trackers.append(tracker)

    # Update the tracker for each car and draw the bounding box
    for tracker in trackers:
        ok, bbox = tracker.update(frame)
        if ok:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the result
    cv2.imshow('Frame', frame)

    # Update the background image for the next frame
    bg = gray

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
!pip uninstall opencv-python
!pip uninstall opencv-contrib-python
!pip uninstall opencv-contrib-python-headle

Found existing installation: opencv-contrib-python 4.5.5.62
Uninstalling opencv-contrib-python-4.5.5.62:
  Would remove:
    /Users/user/opt/anaconda3/lib/python3.8/site-packages/cv2/*
    /Users/user/opt/anaconda3/lib/python3.8/site-packages/opencv_contrib_python-4.5.5.62.dist-info/*
  Would not remove (might be manually added):
    /Users/user/opt/anaconda3/lib/python3.8/site-packages/cv2/.DS_Store
Proceed (y/n)? 

In [ ]:
import cv2

cap = cv2.VideoCapture('../../Downloads//Exercise1_Files/Traffic_Laramie_2.mp4')

# Create background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Capture the first frame and use it as the background image
_, bg = cap.read()
height, width , _ = bg.shape

x_offset = 150
y_offset = 170
x_start_point = int(width/2) - x_offset
x_end_point = int(width/2 )
y_start_point = int(height/2)
y_end_point = int(height/2) + y_offset
bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
tracker = cv2.legacy.TrackerCSRT_create()
tracker.init(bg, (x_start_point, y_start_point, x_offset, y_offset))


bg = bg[y_start_point: y_end_point,x_start_point:x_end_point]

# Initialize list to hold trackers for each car
trackers = cv2.legacy.MultiTracker_create()
my_tracker = cv2.legacy.TrackerCSRT_create()


prev_loc = []
curr_loc = []
count = 0 
while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width , _ = frame.shape
    city_center_street = frame[int(height/2): (int(height/2) + y_offset), int(width/2) - x_offset:int(width/2 )]
    
    # Convert the current frame to grayscale
    gray = cv2.cvtColor(city_center_street, cv2.COLOR_BGR2GRAY)
    
    # Perform background subtraction to obtain the foreground mask
    fgmask = fgbg.apply(gray)

    # Perform frame differencing to obtain the difference mask
    diff = cv2.absdiff(gray, bg)

    # Combine the foreground mask and the difference mask to get the final mask
    mask = cv2.bitwise_and(fgmask, diff)

    # Find contours in the final mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    curr_loc = []
    x = y = w = h= 0
   

    # Loop over the contours to filter the cars and create bounding boxes
    for cnt in contours:
        if cv2.contourArea(cnt) < 2000:
            continue
            
        x, y, w, h = cv2.boundingRect(cnt)
       
        area = w * h
        aspect_ratio = w / h
        if area > 6500 and aspect_ratio > 0.3 and aspect_ratio < 3:

            x=  x + x_start_point
            y = y + y_start_point
           
            cv2.rectangle(frame, (x, y ), (x+w, y+h), (0,255,0), 1)
            curr_loc.extend((x,y))
            
    if len(prev_loc) > 0 and len(curr_loc) == 0:
        print('car passed')
        count+=1

    ok, bbox = tracker.update(frame)

           
    cv2.rectangle(frame, (int((x_start_point + x_end_point)/2), y_start_point  ), (int((x_start_point + x_end_point)/2), y_end_point), (0,255,0), 1)
    # Show the result
    cv2.imshow('Frame', frame)
    cv2.imshow(' Counting point', city_center_street)
    cv2.putText(frame, f"Cars passed: {count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Update the background image for the next frame
    bg = gray
    prev_loc = curr_loc.copy()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindos()




In [ ]:
# Import math Library
import math 

p = [3] 
q = [1] 

# Calculate Euclidean distance
print (math.dist(p, q))

p = [5,1] 
q = [0, 12] 

# Calculate Euclidean distance
print (math.dist(p, q))




In [ ]:
test = []
test.extend((3,4))
print(test)